In [1]:
%matplotlib inline

import numpy as np
from IPython.display import Image

import wind
from PlotInterface.maps import WindfieldMap, BarbMapFigure, levels, FilledContourMapFigure

from Utilities.config import ConfigParser
from Utilities.tracks2shp import tracks2line, tracks2point

configFile = "C:/WorkSpace/data/TC/config/cairns.ini"
trackFile = "C:/WorkSpace/data/TC/cairns/tracks/tracks.00133.nc"
config = ConfigParser()
config.read(configFile)

    
tracks = wind.loadTracks(trackFile)
print len(tracks)
wfg = wind.WindfieldGenerator(config=config)


In [2]:
figure = FilledContourMapFigure()
lvls, exp = levels(100, 0)
cbarlab = "Wind speed (m/s)"

for track in tracks:
    wfg.setGridLimit(track)
    wt = wind.WindfieldAroundTrack(track, windFieldType='kepert',margin=5.0, resolution=0.02)
    result = wt.regionalExtremes(wfg.gridLimit)
    gust, bearing, Vx, Vy, P, lon, lat = result
    [xgrid, ygrid] = np.meshgrid(lon, lat)
    map_kwargs = dict(llcrnrlon=lon.min(),
                  llcrnrlat=lat.min(),
                  urcrnrlon=lon.max(),
                  urcrnrlat=lat.max(),
                  projection='merc',
                  resolution='i')

    figure.add(gust, xgrid, ygrid, "{0}".format(track.trackId), lvls, cbarlab, map_kwargs)
    
figure.plot()
figure.save("windfield.png")
Image("windfield.png")

In [2]:
tracks[1].data

In [3]:
from netCDF4 import num2date

print num2date(tracks[1].data['Datetime'],'hours since 1900-01-01 00:00' )

In [ ]:
tracks[1].data['Datetime'] = num2date(tracks[1].data['Datetime'], 'hours since 1900-01-01 00:00')

In [14]:
dir(tracks[1].data['Datetime'][0])

In [12]:
header = 'CycloneNumber,Datetime,TimeElapsed,Longitude,' + \
                 'Latitude,Speed,Bearing,' + \
                 'CentralPressure,EnvPressure,rMax\n'
fmt = '%i,%s,%7.3f,%8.3f,%8.3f,%6.2f,%6.2f,%7.2f,%7.2f,%6.2f'

In [14]:
trackFile = "C:/WorkSpace/data/TC/cairns/scenario2.csv"
fp = open(trackFile, 'w')
np.savetxt(fp, tracks[1].data, fmt=fmt)
fp.close()